#### The code is expanding the work available in: https://github.com/pucrs-automated-planning/pddl-parser , and; Specific snippets (for heuristics and informed (Dijkstra, A*, Greedy Best First) and uninformed (DFS, BFS) search algorithms) can be also mutated from: https://github.com/APLA-Toolbox/PythonPDDL

#### Check URL: https://github.com/remykarem/py2pddl for dynamic PDDL-based planning (Dynamic Python to Dynamic PDDL regeneration)

In [1]:
### Install a pip package (e.g., numpy) in the current Jupyter kernel
#import sys
#!{sys.executable} -m pip install numpy

## PDDL Parser (only):

In [2]:
# running the PDDL.py main file from the terminal using python
!python PDDL.py # examples/dwr/dwr.pddl examples/dwr/pb1.pddl

Domain name: dwr
action: move
  parameters: [['?r', 'robot'], ['?from', 'location'], ['?to', 'location']]
  positive_preconditions: [['adjacent', '?from', '?to'], ['at', '?r', '?from']]
  negative_preconditions: [['occupied', '?to']]
  add_effects: [['occupied', '?to'], ['at', '?r', '?to']]
  del_effects: [['occupied', '?from'], ['at', '?r', '?from']]

action: load
  parameters: [['?k', 'crane'], ['?r', 'robot'], ['?c', 'container'], ['?l', 'location']]
  positive_preconditions: [['at', '?r', '?l'], ['holding', '?k', '?c'], ['belong', '?k', '?l'], ['unloaded', '?r']]
  negative_preconditions: [['equal', '?c', 'pallet']]
  add_effects: [['loaded', '?r', '?c'], ['empty', '?k']]
  del_effects: [['holding', '?k', '?c'], ['unloaded', '?r']]

action: unload
  parameters: [['?k', 'crane'], ['?r', 'robot'], ['?c', 'container'], ['?l', 'location']]
  positive_preconditions: [['at', '?r', '?l'], ['loaded', '?r', '?c'], ['belong', '?k', '?l'], ['empty', '?k']]
  negative_preconditions: [['equal',

## PDDL Planner (baseline BFS planner):

In [3]:
!python -B planner.py examples/dinner/dinner.pddl examples/dinner/pb1.pddl -v

Time: 0.000462055206299s
plan:
action: cook
  parameters: []
  positive_preconditions: [['clean']]
  negative_preconditions: []
  add_effects: [['dinner']]
  del_effects: []

action: wrap
  parameters: []
  positive_preconditions: [['quiet']]
  negative_preconditions: []
  add_effects: [['present']]
  del_effects: []

action: carry
  parameters: []
  positive_preconditions: [['garbage']]
  negative_preconditions: []
  add_effects: []
  del_effects: [['garbage'], ['clean']]



In [4]:
def pause(variable): # function to pause and print values in the console 
    print("********************************")
    print(f"Debug: \n{variable}\n")
    wait = input("Press Enter to continue!")
    print("********************************")

In [5]:
def validate_conditions(state, positive, negative): # function to validate if positive and negative conditions are valid at a given state
        return positive.issubset(state) and negative.isdisjoint(state)

In [6]:
def state_transition(state, positive, negative): # function to apply the transition of the state (activate positive and deactivate negative effects)
        return state.difference(negative).union(positive)

In [7]:
def solve_BFS(parser):
        
        # Parsed data (all three objects are type: frozenset i.e., immutable static objects)
        state = parser.state # initial problem.pddl state
        goal_pos = parser.positive_goals # goal state positive conditions
        goal_not = parser.negative_goals # goal state negative conditions
        
        # Check if the goal state has been reached (no planning is required)
        if validate_conditions(state, goal_pos, goal_not):
            print('Initial state meets the goal conditions!')
            return []
        
        # Grounding all actions to generate every valid instantiation (based on the number and types of objects defined in problem.pddl)
        grounded_actions = []
        for action in parser.actions:
            for act in action.groundify(parser.objects, parser.types):
                # parser.objects are all instantiated objects (e.g., crane: [a, b, c, d])
                # parser.types are all different types of instantiated objects (e.g., crane)
                grounded_actions.append(act)
                       
        # Graph Search
        closed_set = set([state]) # we already checked if the initial state is the goal state so we consider it in the closed_set
        frontier_set = [state, None] # frontier_set is a list of: [ state, (action that led to this state, plan that led to this state from the root) ]
        
        while frontier_set: # while frontier_set is not empty
            
            # implementing Breadth-First-Non-Informed-Search (BFS) where the visited/closed states set is served in FIFO manner
            # since the state under consideration is popped(0) which is the oldest of the appended 'frontier' set 'new_states'
            state = frontier_set.pop(0) # popping out (remove and assign) the first (or appended) state from the frontier_set
            plan = frontier_set.pop(0) # popping out (remove and assign) the first (or appended) plan from the frontier_set i.e., None
            
            for act in grounded_actions: # iterate over all grounded actions
                
                if validate_conditions(state, act.positive_preconditions, act.negative_preconditions): # check if the grounded action is currently applicable
                    new_state = state_transition(state, act.add_effects, act.del_effects)
                    
                    if new_state not in closed_set: # check if the new_state has not been visited/evaluated already
                        if validate_conditions(new_state, goal_pos, goal_not): # check if the new state is the goal state
                            
                            full_plan = [ act ] # initialise the full plan with the last action
                            while plan:
                                act, plan = plan # iteratively unfolding the enveloped plan and the respective sequence of actions
                                full_plan.insert(0, act) # populate the full_plan 0-position entry with the previous action i.e., the sequence of actions led to the goal state

                            return full_plan
                        
                        closed_set.add(new_state) # appending (at the end of the list) the new_state (which is not the goal state)
                        frontier_set.append(new_state) # appending (at the end of the list) the new_state (which is not the goal state)
                        frontier_set.append((act, plan)) # appending the grounded action led to the new_state alongside with the up-to-date plan(t) = [ action(t), plan(t-1) ]
                    
        return None # return 'None' if the goal state has not been reached

In [8]:
def export_plan(plan, verbose = True):
    
    print('\n----------------------------')
    if type(plan) is list:
        print('Plan:')
        for act in plan:
            print(act if verbose else act.name + ' ' + ' '.join(act.parameters))
    else:
        print('No plan was found')
        exit(1)

In [9]:
def export_parser(parser):
    
    print('\n----------------------------')
    print('Domain name: ' + parser.domain_name)
    for act in parser.actions:
        print(act)
    print('Problem name: ' + parser.problem_name)
    print('Objects: ' + str(parser.objects))
    print('Types: ' + str(parser.types))
    print('Initial State: ' + str(parser.state))
    print('Positive goal conditions: ' + str(parser.positive_goals))
    print('Negative goal conditions: ' + str(parser.negative_goals))

In [10]:
def generate_plan(parser):
    #export_parser(parser)
    print('Parse Time: ' + str(parse_time-start_time) + 's\n')

    # Planner instantiation
    plan = solve_BFS(parser)
    plan_time = time.time()

    #export_plan(plan)
    print('Plan Time: ' + str(plan_time-parse_time) + 's\n')
    
    return plan

## Generating the initial plan

In [11]:
from PDDL import PDDL_Parser
import time

all_plans = []
domain = "examples/dwr/dwr.pddl"
problem = "examples/dwr/pb1.pddl"

start_time = time.time()
# Parser instantiation
parser = PDDL_Parser()
parser.parse_domain(domain)
parser.parse_problem(problem)
parse_time = time.time()

# Generate plan
active_plan = generate_plan(parser)
all_plans.append(active_plan)

Parse Time: 0.002398967742919922s

Plan Time: 2.291515827178955s



## Creating a dynamic re-planning framework

In [13]:
# function for advancing the system state by applying 'apply_plan_steps_to_replan' of the 'plan' action sequence
def progress_system_state(parser, plan, simulation_step, apply_plan_steps_to_replan=1):
    if apply_plan_steps_to_replan > len(plan):
        apply_plan_steps_to_replan = len(plan)

    if type(plan) is list:
        for step in range(0,apply_plan_steps_to_replan):
            act = plan[step]
            simulation_step += 1
            print(f'\nSimulation Step: {simulation_step}\n')
            print('Applying Planned Action:')
            print(act)
            new_state = state_transition(parser.state, act.add_effects, act.del_effects)
            parser.state = new_state

        print('Reached State:')
        print(parser.state)
        
    return parser, simulation_step

In [ ]:
def impose_stochastic_disturbance(parser, type_of_disturbance='fact')
    # iterate the solve(domain, problem) function, over the optimal plan states / actions / steps changing different aspects of the deterministic environment evolution:
    # e.g., assume that the plan will be applied and change anything at the 2nd node/state of the plan in order to re-plan:
    # different initial state (consider the refined 2nd node state as the initial one) -> use state_transition() function;
    # different number of same type objects (simulate robot failures or new grid obstacles/blockades from accidents) -> extend parser.objects entries appropriately
    # different goals (simulate dynamically changing goal) -> extend / refine the goal_pos and goal_not conditions appropriately

    return parser

In [14]:
apply_steps_until_next_replan = 1 # e.g., apply applied_plan_steps_to_replan=1 step(s) of the currently optimized plan and then replan
number_of_replanning_attempts = 100 # e.g., number of attempts to replan after 'apply_plan_steps_to_replan'
simulation_step = 0

number_of_replanning_attempts = min(number_of_replanning_attempts, len(active_plan)) # making sure than the replanning attempts are less than the initial plans length
for replans in range(number_of_replanning_attempts):
    if active_plan:
        parser, simulation_step = progress_system_state(parser, active_plan, simulation_step, apply_steps_until_next_replan)
        parser = impose_disturbance(parser, 'fact')
        print(f'Attempt to replan: {replans}')
        active_plan = generate_plan(parser)
        all_plans.append(active_plan) # replanning after applying actions based on the previously generated plan


Simulation Step: 1

Applying Planned Action:
action: take
  parameters: ('k1', 'cc', 'cb', 'p1', 'l1')
  positive_preconditions: [['belong', 'k1', 'l1'], ['on', 'cc', 'cb'], ['empty', 'k1'], ['attached', 'p1', 'l1'], ['in', 'cc', 'p1'], ['top', 'cc', 'p1']]
  negative_preconditions: [['equal', 'cc', 'cb'], ['equal', 'cc', 'pallet']]
  add_effects: [['top', 'cb', 'p1'], ['holding', 'k1', 'cc']]
  del_effects: [['on', 'cc', 'cb'], ['empty', 'k1'], ['top', 'cc', 'p1'], ['in', 'cc', 'p1']]

Reached State:
frozenset({('adjacent', 'l2', 'l1'), ('top', 'pallet', 'q2'), ('unloaded', 'r1'), ('empty', 'k2'), ('in', 'ca', 'p1'), ('occupied', 'l1'), ('top', 'cb', 'p1'), ('top', 'pallet', 'p2'), ('attached', 'q1', 'l1'), ('belong', 'k1', 'l1'), ('in', 'cf', 'q1'), ('equal', 'ce', 'ce'), ('equal', 'cd', 'cd'), ('in', 'cb', 'p1'), ('at', 'r1', 'l1'), ('on', 'cf', 'ce'), ('on', 'ce', 'cd'), ('in', 'cd', 'q1'), ('equal', 'cc', 'cc'), ('attached', 'p1', 'l1'), ('attached', 'q2', 'l2'), ('belong', 'k2',

Plan Time: 9.125623941421509s


Simulation Step: 8

Applying Planned Action:
action: load
  parameters: ('k1', 'r1', 'cb', 'l1')
  positive_preconditions: [['belong', 'k1', 'l1'], ['unloaded', 'r1'], ['at', 'r1', 'l1'], ['holding', 'k1', 'cb']]
  negative_preconditions: [['equal', 'cb', 'pallet']]
  add_effects: [['loaded', 'r1', 'cb'], ['empty', 'k1']]
  del_effects: [['holding', 'k1', 'cb'], ['unloaded', 'r1']]

Reached State:
frozenset({('adjacent', 'l2', 'l1'), ('top', 'pallet', 'q2'), ('empty', 'k2'), ('in', 'ca', 'p1'), ('occupied', 'l1'), ('attached', 'q1', 'l1'), ('belong', 'k1', 'l1'), ('in', 'cf', 'q1'), ('equal', 'ce', 'ce'), ('equal', 'cd', 'cd'), ('at', 'r1', 'l1'), ('on', 'cf', 'ce'), ('on', 'ce', 'cd'), ('top', 'cc', 'p2'), ('in', 'cd', 'q1'), ('equal', 'cc', 'cc'), ('on', 'cc', 'pallet'), ('attached', 'p1', 'l1'), ('attached', 'q2', 'l2'), ('belong', 'k2', 'l2'), ('on', 'cd', 'pallet'), ('equal', 'pallet', 'pallet'), ('on', 'ca', 'pallet'), ('adjacent', 'l1', 'l2'), ('l

Plan Time: 10.111098051071167s


Simulation Step: 16

Applying Planned Action:
action: unload
  parameters: ('k2', 'r1', 'ca', 'l2')
  positive_preconditions: [['empty', 'k2'], ['loaded', 'r1', 'ca'], ['at', 'r1', 'l2'], ['belong', 'k2', 'l2']]
  negative_preconditions: [['equal', 'ca', 'pallet']]
  add_effects: [['unloaded', 'r1'], ['holding', 'k2', 'ca']]
  del_effects: [['empty', 'k2'], ['loaded', 'r1', 'ca']]

Reached State:
frozenset({('adjacent', 'l2', 'l1'), ('unloaded', 'r1'), ('on', 'cb', 'pallet'), ('attached', 'q1', 'l1'), ('at', 'r1', 'l2'), ('belong', 'k1', 'l1'), ('in', 'cf', 'q1'), ('equal', 'ce', 'ce'), ('equal', 'cd', 'cd'), ('on', 'cf', 'ce'), ('holding', 'k2', 'ca'), ('on', 'ce', 'cd'), ('top', 'cc', 'p2'), ('in', 'cd', 'q1'), ('equal', 'cc', 'cc'), ('on', 'cc', 'pallet'), ('attached', 'p1', 'l1'), ('attached', 'q2', 'l2'), ('belong', 'k2', 'l2'), ('on', 'cd', 'pallet'), ('top', 'cb', 'q2'), ('equal', 'pallet', 'pallet'), ('top', 'pallet', 'p1'), ('adjacent', 'l1', '

In [15]:
## replanning after applying actions from the previously generated plan
#active_plan = generate_plan(parser)
#all_plans.append(active_plan) # replanning after applying actions based on the previously generated plan

0